### Creation of the environment

In [1]:
import os
os.environ['USE_AUTH_EPHEM'] = '0'

In [2]:
%tensorflow_version 2.x
!pip3 install --upgrade pip
#!pip install -qU t5
!pip3 install git+https://github.com/google-research/text-to-text-transfer-transformer.git #extra_id_x support

import functools
import os
import time
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)

import tensorflow.compat.v1 as tf
tf.flags.DEFINE_string('f','','')

import tensorflow_datasets as tfds

import t5


#Set the base dir(Google cloud bucket)
BASE_DIR = "gs://code-generation" 

#C.B.: SEQ_LENGTH for number of tokens
SEQ_LENGTH = 512

if not BASE_DIR or BASE_DIR == "gs://":
  raise ValueError("You must enter a BASE_DIR.")
ON_CLOUD = True


if ON_CLOUD:
  import tensorflow_gcs_config
  from google.colab import auth
  # Set credentials for GCS reading/writing from Colab and TPU.
  TPU_TOPOLOGY = "2x2"
  try:
    tpu = tf.distribute.cluster_resolver.TPUClusterResolver()  # TPU detection
    TPU_ADDRESS = tpu.get_master()
    print('Running on TPU:', TPU_ADDRESS)
  except ValueError:
    raise BaseException('ERROR: Not connected to a TPU runtime; please see the previous cell in this notebook for instructions!')
  auth.authenticate_user()
  tf.config.experimental_connect_to_host(TPU_ADDRESS)
  tensorflow_gcs_config.configure_gcs_from_colab_auth()

tf.disable_v2_behavior()

# Improve logging.
from contextlib import contextmanager
import logging as py_logging

if ON_CLOUD:
  tf.get_logger().propagate = False
  py_logging.root.setLevel('INFO')

@contextmanager
def tf_verbosity_level(level):
  og_level = tf.logging.get_verbosity()
  tf.logging.set_verbosity(level)
  yield
  tf.logging.set_verbosity(og_level)

     |████████████████████████████████| 2.1 MB 5.0 MB/s 
  Attempting uninstall: pip
    Found existing installation: pip 21.1.3
    Uninstalling pip-21.1.3:
      Successfully uninstalled pip-21.1.3
  Cloning https://github.com/google-research/text-to-text-transfer-transformer.git to /tmp/pip-req-build-hbwg8uz6
  Running command git clone --filter=blob:none --quiet https://github.com/google-research/text-to-text-transfer-transformer.git /tmp/pip-req-build-hbwg8uz6
  Resolved https://github.com/google-research/text-to-text-transfer-transformer.git to commit b7d861c6b383c97eda6de7dfe5d2867a3c6afd0a
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 385.2/385.2 kB 9.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 90.7/90.7 kB 7.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 44.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 301.2/301.2 kB 27.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━

Instructions for updating:
non-resource variables are not supported in the long term


### Loading of tsv files
With this script you can load each tsv file for finetuning.
Please be sure that the path to all tsv files are correct

In [3]:
train_construct_length = 251271
test__construct_length = 37722

train_block_length = 101646
test__block_length = 12706

train_token_length = 307779
test__token_length = 10930

In [4]:
#Validation(train and test on the same dataset)

nq_tsv_path_construct = {
    "train":      BASE_DIR + '/T5_extension/ft_datasets/construct_train.tsv',
    "validation": BASE_DIR + '/T5_extension/ft_datasets/construct_test.tsv',
}

num_nq_examples_construct = dict(train=train_construct_length, validation=test__construct_length)

In [5]:
#Validation(train and test on the same dataset)

nq_tsv_path_block = {
    "train":      BASE_DIR + '/T5_extension/ft_datasets/block_train.tsv',
    "validation": BASE_DIR + '/T5_extension/ft_datasets/block_test.tsv',
}

num_nq_examples_block = dict(train=train_block_length, validation=test__block_length)

In [6]:
#Validation(train and test on the same dataset)

nq_tsv_path_token = {
    "train":      BASE_DIR + '/T5_extension/ft_datasets/token_train.tsv',
    "validation": BASE_DIR + '/T5_extension/ft_datasets/token_test.tsv',
}

num_nq_examples_token = dict(train=train_token_length, validation=test__token_length)

### Preprocess of the dataset
In this step we preprocess the dataset.  
You have to change the path to vocab files (*vocab_model_path* and *vocab_path*)
We're going to preprocess all the tsv file so that T5 can use them for evaluation.

In [7]:
from t5.data import postprocessors as t5_postprocessors
from t5.seqio import Feature,SentencePieceVocabulary


# # Set the path of sentencepiece model and vocab files
# # Must be the same used for the pre-trained phase
vocab_model_path = BASE_DIR + '/T5_extension/code.model'
vocab_path = BASE_DIR + '/T5_extension/code.vocab'


TaskRegistry = t5.data.TaskRegistry
TfdsTask = t5.data.TfdsTask


def get_default_vocabulary():
  return SentencePieceVocabulary(vocab_model_path, 100)

DEFAULT_OUTPUT_FEATURES = {
    "inputs": Feature(
        vocabulary=get_default_vocabulary(), add_eos=True, required=False),

    "targets": Feature(
        vocabulary=get_default_vocabulary(), add_eos=True)
}

JAVA CONSTRUCT

In [8]:
def nq_construct(split, shuffle_files=True):
  # We only have one file for each split.
  del shuffle_files

   # Load lines from the text file as examples.

  ds = tf.data.TextLineDataset(nq_tsv_path_construct[split])
  ds = ds.map(
      functools.partial(tf.io.decode_csv, record_defaults=["string","string"],
                        field_delim="\t", use_quote_delim=False),
      num_parallel_calls=tf.data.experimental.AUTOTUNE)
  
  ds = ds.map(lambda *ex: dict(zip(["input", "output"], ex)))
  return ds

print("A few raw train examples...")
for ex in tfds.as_numpy(nq_construct("train").take(5)):
  print(ex)

A few raw train examples...
{'input': b'private String formatEventDateRange(Date beginDate, Date endDate) { if ( <extra_id_0>) { if (isEndOfDay(endDate)) { return formatEventDate(beginDate); } else if (isMidnight(beginDate)) { return formatEventDate(beginDate) + " until " + eventOutDayOnlyDf.format(endDate); } else { return formatEventDate(beginDate) + " - " + eventOutTimeOnlyDf.format(endDate); } } else { return formatEventDate(beginDate) + " - " + formatEventDate(endDate); } } <CONST> RSSItem(String, String, String, String, String, String, String, String) <INV> String formatEventDate(Date), boolean isMidnight(Date), boolean isEndOfDay(Date) <OTH> String sanitizeString(String), String sanitizeStringWithTags(String), String getTitle(), String getDescription(), String getLink(), String getAuthor(), String getDate(), URL getImgUrl(), String toString()', 'output': b'DateUtils.isSameDay(beginDate, endDate)'}
{'input': b'private String formatEventDateRange(Date beginDate, Date endDate) { if

In [9]:
def construct_preprocessing(ds):
  
  def to_inputs_and_targets(ex):

        inputs = tf.strings.join(['CONSTRUCT:' + ex['input']], separator=' ')
        class_label = tf.strings.join([ex['output']], separator=' ')
        return {'inputs': inputs, 'targets': class_label }
    
  return ds.map(to_inputs_and_targets, 
                num_parallel_calls=tf.data.experimental.AUTOTUNE)

In [10]:
t5.data.TaskRegistry.remove('construct')
t5.data.TaskRegistry.add(
    "construct",
    dataset_fn=nq_construct,
    splits=["train", "validation"],
    text_preprocessor=[construct_preprocessing],
    output_features = DEFAULT_OUTPUT_FEATURES,
    metric_fns=[t5.evaluation.metrics.accuracy],
    num_input_examples=num_nq_examples_construct
)

In [11]:
nq_task = t5.data.TaskRegistry.get("construct")
ds = nq_task.get_dataset(split="train", sequence_length={"inputs": SEQ_LENGTH, "targets": SEQ_LENGTH})
print("A few preprocessed training examples...")
for ex in tfds.as_numpy(ds.take(5)):
  print(ex)


A few preprocessed training examples...
{'inputs_pretokenized': b'CONSTRUCT:public void setupHomeActivity() { super.setupHomeActivity(); if ( <extra_id_0>) { mActivity.getActionBar().setDisplayOptions( 0, ActionBar.DISPLAY_SHOW_HOME | ActionBar.DISPLAY_SHOW_TITLE); } else { mActivity.getActionBar().setDisplayOptions( ActionBar.DISPLAY_USE_LOGO, ActionBar.DISPLAY_USE_LOGO | ActionBar.DISPLAY_SHOW_TITLE); } } <CONST> ActivityHelperHoneycomb(Activity) <INV> <OTH> void onPostCreate(Bundle), boolean onCreateOptionsMenu(Menu), boolean onOptionsItemSelected(MenuItem), void setupSubActivity(), void setActionBarTitle(CharSequence), void setActionBarColor(int), void setRefreshActionButtonCompatState(boolean)', 'inputs': array([12094, 30813,    78,  2825,    65,  2193,  4462,  5733,  1927,
          20,   436,    12,  2060,   627,  4462,  5733, 12329,    56,
           4, 32099,    91,    20,   101,  5733,    12,  1323,  1001,
        5099,  1927,    12,  2060,  5772,  1044,   451,    88,     9,


JAVA TOKEN

In [12]:
def nq_token(split, shuffle_files=False):
  # We only have one file for each split.
  del shuffle_files

  # Load lines from the text file as examples.
  ds = tf.data.TextLineDataset(nq_tsv_path_token[split])
  ds = ds.map(
      functools.partial(tf.io.decode_csv, record_defaults=["string","string"],
                        field_delim="\t", use_quote_delim=False),
      num_parallel_calls=tf.data.experimental.AUTOTUNE)
  
  ds = ds.map(lambda *ex: dict(zip(["input", "output"], ex)))
  return ds

print("A few raw valid examples...")
for ex in tfds.as_numpy(nq_token("validation").take(5)):
  print(ex)

A few raw valid examples...
{'input': b'public static Analyzer fromSpec(final JSONObject json, final String analyzerKey) throws JSONException { JSONObject spec = json.optJSONObject(analyzerKey<extra_id_0> if (spec != null) { return getAnalyzer(spec); } else { return getAnalyzer(json.optString(analyzerKey, Constants.DEFAULT_ANALYZER)); } }', 'output': b');'}
{'input': b'public static Analyzer getAnalyzer(final String str) throws JSONException { final String[] parts = str.split(":", 2); final String name = parts[0].toUpperCase(); final String args = parts.length == 2 <extra_id_0> return Analyzers.valueOf(name).newAnalyzer(args); }', 'output': b'? parts[1] : null;'}
{'input': b'private static List<ParamSpec> getParamSpecs(JSONArray jsonParams) throws ParameterException, JSONException { final List<ParamSpec> paramSpecs = new ArrayList<>(); if (jsonParams != null) { for (int i = 0; i < jsonParams.length()<extra_id_0> paramSpecs.add(getParamSpec(jsonParams.getJSONObject(i))); } } return para

In [13]:
def token_preprocessing(ds):
  
  def to_inputs_and_targets(ex):

        inputs = tf.strings.join(['TOKEN:' + ex['input']], separator=' ')
        class_label = tf.strings.join([ex['output']], separator=' ')
        return {'inputs': inputs, 'targets': class_label }
    
  return ds.map(to_inputs_and_targets, 
                num_parallel_calls=tf.data.experimental.AUTOTUNE)

In [14]:
t5.data.TaskRegistry.remove('token')
t5.data.TaskRegistry.add(
    "token",
    dataset_fn=nq_token,
    splits=["train", "validation"],
    text_preprocessor=[token_preprocessing],
    output_features = DEFAULT_OUTPUT_FEATURES,
    metric_fns=[t5.evaluation.metrics.accuracy],
    num_input_examples=num_nq_examples_token
)

In [15]:
nq_task = t5.data.TaskRegistry.get("token")
ds = nq_task.get_dataset(split="train", sequence_length={"inputs": SEQ_LENGTH, "targets": SEQ_LENGTH})
print("A few preprocessed training examples...")
for ex in tfds.as_numpy(ds.take(5)):
  print(ex)


A few preprocessed training examples...
{'inputs_pretokenized': b'TOKEN:public void setPosicaoTexto(String ver, <extra_id_0> posVer = ver; posHor = hor; atualizaDataTexto(); } <CONST> BotaoBase(PApplet, PVector, float, int) <INV> void atualizaDataTexto() <OTH> void setImageBotao(String), void setPosBotaoMenor(PVector), void setDiamMen(float), void setOffsetToPosII(PVector), void setPosicaoDefault(boolean), boolean getPosDefault(), void setTamanhoTexto(float), void setColorOff(int), void setColorON(int), void setColorFixo(int), void setBotaoComoToggle(), void desenhaTexto(), void desenhaTextoYOffset(float), void desenhaTextoGira(float), void setNomeBotao(String), void setEtiqueta(String), void setNovaPos(PVector), void desenharBotaoCircular(), void desenharBotaoCircular(boolean), void desenharBotaoCircularMovil(), void desenharBotaoCircularMovil(boolean), void desenharBotaoCircularTextoGira(float), void setTamanhoEtiqueta(float), void desenharBotaoCircularSemTexto(), boolean botaoToogle

JAVA BLOCK

In [16]:
def nq_block(split, shuffle_files=False):
  # We only have one file for each split.
  del shuffle_files

  # Load lines from the text file as examples.
  ds = tf.data.TextLineDataset(nq_tsv_path_block[split])
  ds = ds.map(
      functools.partial(tf.io.decode_csv, record_defaults=["string","string"],
                        field_delim="\t", use_quote_delim=False),
      num_parallel_calls=tf.data.experimental.AUTOTUNE)
  
  ds = ds.map(lambda *ex: dict(zip(["input", "output"], ex)))
  return ds

print("A few raw valid examples...")
for ex in tfds.as_numpy(nq_block("validation").take(5)):
  print(ex)

A few raw valid examples...
{'input': b'public void removeItem(int position, boolean argIsSilent) { if (position < 0) <extra_id_0> int size = mInternalPlaylist.size(); if (size > position) { mInternalPlaylist.remove(position); } if (!argIsSilent) notifyPlaylistChanged(); } <CONST> Playlist(Context, int), Playlist(Context, int, boolean), Playlist(Context) <INV> int size(), void notifyPlaylistChanged(), void removeItem(int) <OTH> void setContext(Context), void destroy(), ArrayList getPlaylist(), int defaultSize(), IEpisode getItem(int), IEpisode getNext(), int getPosition(IEpisode), void setAsFrist(IEpisode), void setItem(int, IEpisode), boolean maybeInsert(IEpisode), boolean insertEpisodeInternal(IEpisode), void removeFirst(), IEpisode nextEpisode(), void move(int, int), void queue(IEpisode), List getMediaItems(PlayerService), String getOrder(), String getWhere(), void resetPlaylist(CursorAdapter), boolean populatePlaylistIfEmpty(), void populatePlaylist(), void populatePlaylist(int, bo

In [17]:
def block_preprocessing(ds):
  
  def to_inputs_and_targets(ex):

        inputs = tf.strings.join(['BLOCK:' + ex['input']], separator=' ')
        class_label = tf.strings.join([ex['output']], separator=' ')
        return {'inputs': inputs, 'targets': class_label }
    
  return ds.map(to_inputs_and_targets, 
                num_parallel_calls=tf.data.experimental.AUTOTUNE)

In [18]:
t5.data.TaskRegistry.remove('block')
t5.data.TaskRegistry.add(
    "block",
    dataset_fn=nq_block,
    splits=["train", "validation"],
    text_preprocessor=[block_preprocessing],
    output_features = DEFAULT_OUTPUT_FEATURES,
    metric_fns=[t5.evaluation.metrics.accuracy],
    num_input_examples=num_nq_examples_block
)

In [19]:
nq_task = t5.data.TaskRegistry.get("block")
ds = nq_task.get_dataset(split="train", sequence_length={"inputs": SEQ_LENGTH, "targets": SEQ_LENGTH})
print("A few preprocessed training examples...")
for ex in tfds.as_numpy(ds.take(5)):
  print(ex)


A few preprocessed training examples...
{'inputs_pretokenized': b'BLOCK:public void onResume() { updateSize(false); if (mCursorBlink != 0) { mHandler.postDelayed(mBlinkCursor, CURSOR_BLINK_PERIOD); } if (mKeyListener != null) <extra_id_0> } <CONST> EmulatorView(Context, TermSession, DisplayMetrics), EmulatorView(Context, AttributeSet), EmulatorView(Context, AttributeSet, int) <INV> void updateSize(boolean) <OTH> int createLinks(int), void commonConstructor(Context), void attachSession(TermSession), void setDensity(DisplayMetrics), void onPause(), void setColorScheme(ColorScheme), boolean onCheckIsTextEditor(), InputConnection onCreateInputConnection(EditorInfo), void setImeBuffer(String), boolean getKeypadApplicationMode(), void setExtGestureListener(GestureDetector), int computeVerticalScrollRange(), int computeVerticalScrollExtent(), int computeVerticalScrollOffset(), void initialize(), TermSession getTermSession(), int getVisibleWidth(), int getVisibleHeight(), int getVisibleRows(),

### Evaluation
You can run the evaluation using the following cells.  
Please set the correct path of the variable *MODEL_DIR* (the path to save the pretrained model in)


In [20]:
def _rate_num_input_examples(task):
  if "train" in task.splits:
    return float(task.num_input_examples("train"))
  elif "validation" in task.splits:
    return float(task.num_input_examples("validation"))
  else:
    raise ValueError("Task %s does not have a train or validation split." % (task.name))


t5.data.MixtureRegistry.remove("all_tasks")
t5.data.MixtureRegistry.add(
    "all_tasks",
    ["construct", "token", "block"],
    default_rate=_rate_num_input_examples
     #default_rate=1.0
)

In [21]:
from mesh_tensorflow.transformer.learning_rate_schedules import slanted_triangular
import t5.models

MODEL_SIZE = "small" 

# Set the folder where the checkpoints and all the others information will be writed
MODEL_DIR = BASE_DIR + '/T5_extension/finetuning'

model_parallelism, train_batch_size, keep_checkpoint_max = {
    "small": (1, 256, 16000),
    "base": (2, 128, 8),
    "large": (8, 64, 4),
    "3B": (8, 16, 1),
    "11B": (8, 16, 1)}[MODEL_SIZE]

tf.io.gfile.makedirs(MODEL_DIR)

model = t5.models.MtfModel(
    model_dir=MODEL_DIR,
    tpu=TPU_ADDRESS,
    tpu_topology=TPU_TOPOLOGY,
    model_parallelism=model_parallelism,
    batch_size=train_batch_size,
    learning_rate_schedule = slanted_triangular,
    sequence_length={"inputs": SEQ_LENGTH, "targets": SEQ_LENGTH},
    save_checkpoints_steps=5000,
    keep_checkpoint_max=keep_checkpoint_max if ON_CLOUD else None,
    iterations_per_loop=100,
)

In [22]:
# ## C.B.: Run only to create files.

# model.batch_size = 512
# model.eval(
#     mixture_or_task_name="all_tasks",
#     checkpoint_steps=-1 #evaluate only last checkpoint
# )

In [23]:
# we used model.predict function (setting beam_size)

vocabulary_predict=get_default_vocabulary()

input_file = BASE_DIR + '/T5_extension/finetuning/predict/inputs.txt'
output_file = BASE_DIR + '/T5_extension/finetuning/predict/predictions.txt'

model.predict(input_file=input_file, output_file=output_file,
              checkpoint_steps=-1, beam_size=1, temperature=0.0, keep_top_k=-1, vocabulary=vocabulary_predict)

INFO:root:system_path_file_exists:gs://code-generation/T5_extension/finetuning/operative_config.gin
ERROR:root:Path not found: gs://code-generation/T5_extension/finetuning/operative_config.gin


INFO:tensorflow:Using config: {'_model_dir': 'gs://code-generation/T5_extension/finetuning', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': 5000, '_save_checkpoints_secs': None, '_session_config': graph_options {
  rewrite_options {
    disable_meta_optimizer: true
  }
}
cluster_def {
  job {
    name: "worker"
    tasks {
      key: 0
      value: "10.84.112.146:8470"
    }
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': None, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_experimental_max_worker_delay_secs': None, '_session_creation_timeout_secs': 7200, '_checkpoint_save_graph_def': True, '_service': None, '_cluster_spec': ClusterSpec({'worker': ['10.84.112.146:8470']}), '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': 'grpc://10.84.112.146:8470', '_evaluation_master': 'grpc://10.84.112.146: